In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

2024-10-24 21:53:57.567709: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 21:53:57.697077: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 21:53:58.255948: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-10-24 21:53:58.256002: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

GPU is available


In [2]:
dataset_directory = r"LinkedIn_Dataset.pcl"
dataset = pd.read_pickle(dataset_directory)
dataset

,Intro,Full Name,Workplace,Location,Connections,Photo,Followers,About,Experiences,Number of Experiences,...,Number of Scores,Languages,Number of Languages,Organizations,Number of Organizations,Interests,Number of Interests,Activities,Number of Activities,Label
0,"{'Full Name': 'chenxia (polly) Pei', 'Workplac...",chenxia (polly) Pei,Jiangsu Junyao mainly offer services to cement...,"Wuxi, Jiangsu, China",500,No,717,Our GMH brick has the proprietary intellectual...,"{'0': {'Role': 'Sales', 'Workplace': 'Jiangsu ...",2,...,0,{},0,{},0,"{'0': {'Interest': 'Trina Solar', 'ID': '69648...",4,{'chenxia-pei-80177594': {'Full Name': 'chenxi...,1,1
1,"{'Full Name': 'NEHA CHANDOK', 'Workplace': 'So...",NEHA CHANDOK,Software Analyst,"Noida, Uttar Pradesh, India",500,No,1340,,"{'0': {'Role': 'Software Analyst', 'Workplace'...",1,...,0,"{'Language 0': 'Bangla', 'Language 1': 'Englis...",3,{},0,"{'0': {'Interest': 'Mohamed El-Erian', 'ID': '...",8,{},0,1
2,"{'Full Name': 'Mounika Mungamuri', 'Workplace'...",Mounika Mungamuri,Senior Consultant at Infosys,"Hyderabad, Telangana, India",7,Yes,7,,"{'0': {'Role': 'Senior Consultant', 'Workplace...",2,...,0,{},0,{},0,{},0,{},0,1
3,"{'Full Name': 'Katarina Djuric', 'Workplace': ...",Katarina Djuric,--,"Belgrade, Serbia",0,Yes,0,,"{'0': {'Role': 'Instructor', 'Workplace': 'GE'...",1,...,0,{},0,{},0,"{'0': {'Interest': 'GE', 'ID': '1015', 'Catego...",1,{},0,1
4,"{'Full Name': 'Rachel Lally', 'Workplace': '--...",Rachel Lally,--,"Dublin, County Dublin, Ireland",61,Yes,61,tictok is is the futer,"{'0': {'Role': 'Bartender', 'Workplace': 'O'Su...",1,...,1,{},0,{},0,"{'0': {'Interest': 'Richard Branson', 'ID': 'r...",3,"{'garyltravis': {'Full Name': 'Gary Travis', '...",6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,{'chatGPT': '1. Emily Williams#@Market Researc...,NaN,NaN,NaN,106,Yes,717,{},{'chatGPT': '1. Market Research Analyst#@Eliza...,3,...,0,{},0,{},0,{},3,{},4,11
3596,{'chatGPT': '1. Sarah Johnson#@Director of Mar...,NaN,NaN,NaN,102,Yes,6,{},{'chatGPT': '1. Director of Marketing#@Acme In...,5,...,0,{},0,{},0,{},5,{},3,11
3597,{'chatGPT': '1. Sarah Johnson#@Manager#@Acme I...,NaN,NaN,NaN,435,Yes,10,{'chatGPT': '1. A results-driven and experienc...,{},0,...,0,{},0,{},0,{},6,{},0,11
3598,{'chatGPT': '1. Emma Williams#@Manager and Dir...,NaN,NaN,NaN,280,Yes,34,{'chatGPT': '1. With over 10 years of experien...,{'chatGPT': 'Operations Manager#@ABC Inc.#@Jan...,1,...,0,{'chatGPT': 'English#@Spanish '},2,{},0,{},4,{},4,11


In [10]:
import pandas as pd
import ast
import numpy as np

def parse_chatgpt_content(content):
    """
    Parse content from chatGPT format and return structured data.
    
    Args:
        content (str): Content string with #@ separators
    
    Returns:
        dict: Structured data
    """
    parts = content.split('#@')
    if len(parts) >= 4:
        # Combine title and company for workplace
        title = parts[1].strip()
        company = parts[2].strip()
        workplace = f"{title} at {company}"
        
        return {
            'Full Name': parts[0].strip(),
            'Workplace': workplace,
            'Location': parts[3].strip(),
            'Connections': '500+',  # Default value
            'Photo': 'No',         # Default value
            'Followers': ''        # Default value
        }
    return {
        'Full Name': '',
        'Workplace': '',
        'Location': '',
        'Connections': '',
        'Photo': '',
        'Followers': ''
    }

def clean_chatgpt_field(field, field_name):
    """
    Cleans fields that have chatGPT wrapper format.
    
    Args:
        field: Field that might have chatGPT wrapper
        field_name: Name of the field being cleaned
    
    Returns:
        Various: Cleaned content in appropriate format
    """
    try:
        # Convert to dict if it's a string representation
        if isinstance(field, str):
            field = ast.literal_eval(field)
            
        # Handle chatGPT format
        if isinstance(field, dict) and 'chatGPT' in field:
            content = field['chatGPT']
            
            # Get first entry (split on newline)
            first_entry = content.split('\n')[0]
            
            # Remove number prefix if it exists
            if first_entry.startswith(('1.', '2.', '3.', '4.', '5.')):
                first_entry = first_entry.split('.', 1)[1].strip()
            
            # Special handling for Intro field
            if field_name == 'Intro':
                return parse_chatgpt_content(first_entry)
            elif field_name == 'Full Name':
                return first_entry.split('#@')[0].strip()
            elif field_name == 'Workplace':
                parts = first_entry.split('#@')
                if len(parts) > 2:
                    title = parts[1].strip()
                    company = parts[2].strip()
                    return f"{title} at {company}"
                return ''
            elif field_name == 'Location':
                parts = first_entry.split('#@')
                return parts[3].strip() if len(parts) > 3 else ''
            else:
                # For other fields, just remove #@ tokens
                return first_entry.replace('#@', ' ')
                
        # Handle normal dictionary format
        elif isinstance(field, dict):
            return field
            
    except:
        if field_name == 'Intro':
            return parse_chatgpt_content('')
        return ''
        
    return field if field is not None else ''

def get_column_dtype(column_name):
    """
    Determine the appropriate dtype for a column based on its name.
    
    Args:
        column_name (str): Name of the column
    
    Returns:
        type: Appropriate dtype for the column
    """
    if 'Number of' in column_name:
        return 'Int64'  # Nullable integer type
    if column_name in ['Connections', 'Followers']:
        return 'string'
    return 'string'

def clean_dataset(df):
    """
    Clean all fields in the dataset.
    
    Args:
        df (pd.DataFrame): Input DataFrame
    
    Returns:
        pd.DataFrame: Cleaned DataFrame
    """
    # Ensure all required columns exist
    required_columns = [
        'Intro', 'Full Name', 'Workplace', 'Location', 'Connections', 'Photo', 
        'Followers', 'About', 'Experiences', 'Number of Experiences', 'Educations',
        'Number of Educations', 'Licenses', 'Number of Licenses', 'Volunteering',
        'Number of Volunteering', 'Skills', 'Number of Skills', 'Recommendations',
        'Number of Recommendations', 'Projects', 'Number of Projects', 'Publications',
        'Number of Publications', 'Courses', 'Number of Courses', 'Honors',
        'Number of Honors', 'Scores', 'Number of Scores', 'Languages',
        'Number of Languages', 'Organizations', 'Number of Organizations',
        'Interests', 'Number of Interests', 'Activities', 'Number of Activities',
        'Label'
    ]
    
    # Initialize DataFrame with all required columns and appropriate dtypes
    cleaned_df = pd.DataFrame(columns=required_columns)
    for col in required_columns:
        cleaned_df[col] = pd.Series(dtype=get_column_dtype(col))
    
    # Copy existing data
    for col in df.columns:
        if col in required_columns:
            cleaned_df[col] = df[col].astype(get_column_dtype(col))
    
    # Clean each column
    for column in cleaned_df.columns:
        cleaned_df[column] = cleaned_df[column].apply(
            lambda x: clean_chatgpt_field(x, column)
        )
    
    # Handle Intro field specifically
    if 'Intro' in cleaned_df.columns:
        # Convert Intro column to list of dicts, replacing None/NaN with empty dict
        intro_dicts = cleaned_df['Intro'].apply(
            lambda x: x if isinstance(x, dict) else parse_chatgpt_content('')
        ).tolist()
        
        # Create DataFrame from intro dicts
        intro_df = pd.DataFrame(intro_dicts)
        
        # Update corresponding columns from intro data
        for col in intro_df.columns:
            if col in cleaned_df.columns:
                # Fill NaN values with appropriate default values based on column type
                default_value = 0 if 'Number of' in col else ''
                intro_df[col] = intro_df[col].fillna(default_value)
                cleaned_df[col] = cleaned_df[col].fillna(default_value)
                
                # Convert to appropriate type before updating
                dtype = get_column_dtype(col)
                intro_df[col] = intro_df[col].astype(dtype)
                cleaned_df[col] = cleaned_df[col].astype(dtype)
                
                # Update only where cleaned_df is empty or zero
                if 'Number of' in col:
                    mask = cleaned_df[col].isna() | (cleaned_df[col] == 0)
                else:
                    mask = cleaned_df[col].astype(str).str.strip() == ''
                
                cleaned_df.loc[mask, col] = intro_df.loc[mask, col]
    
    return cleaned_df

def process_and_save_dataset(dataset, output_file):
    """
    Process the dataset and save to CSV.
    
    Args:
        dataset (pd.DataFrame): Input DataFrame
        output_file (str): Path to output CSV file
    """
    # Clean the dataset
    cleaned_df = clean_dataset(dataset)
    
    # Save to CSV
    cleaned_df.to_csv(output_file, index=False, encoding='utf-8')
    return cleaned_df

# Example usage:
cleaned_df = process_and_save_dataset(dataset, 'cleaned_profiles.csv')